# Looping & piping questions
This notebook provides example EDSL code for automatically looping (repeating) a question with content piped from other questions and answers.

Please see the EDSL [documentation page](https://docs.expectedparrot.com/en/latest/surveys.html) for more details on each of the object types and methods for looping questions and piping questions and answers that are used below.

In [1]:
from edsl import QuestionNumerical, Scenario, ScenarioList, Survey

We start by creating an initial question (with no content piped into it).
EDSL comes with many common [question types](https://docs.expectedparrot.com/en/latest/questions.html) that we can choose from based on the form of the response that we want to get back from a model (e.g., free text, multiple choice, linear scale, etc.).
Here we use a numerical question:

In [2]:
q_0 = QuestionNumerical(
    question_name = "q_0",
    question_text = "Please give me a random number.",
    min_value = 1, 
    max_value = 100,
    answering_instructions = "The number must be an integer."
)

Next we create a question that we will "loop" (repeat) some number of times.
We use double braces to create a `{{ placeholder }}` for content to be added to the question when we create copies of it

Here we want to simultaneously set the names of the copies of the question and reference those names in the versions of the question text, so that content from one question and answer can be automatically piped into another cope of the question.
To do this, we create placeholders for each question name (`{{ num }}`) (it must be unique) and question text (`{{ text }}`).
Then in the next step we reference the question names in those texts.

(Note that the names of the placeholders can be anything other than reserved names, and this example works with any other question types as well. We just use a numerical question to keep the responses brief and easy to check!)

In [3]:
q = QuestionNumerical(
    question_name = "q_{{ scenario.num }}",
    question_text = "{{ scenario.text }}",
    min_value = 1, 
    max_value = 100,
    answering_instructions = "The number must be an integer."
)

Next we create a list of `Scenario` objects for the question name and question text inputs that we will pass to the `loop` method that we call on the question in order to create the copies (learn more about [using scenarios](https://docs.expectedparrot.com/en/latest/scenarios.html)):

In [4]:
s = ScenarioList(
    [Scenario({
        "num": n,
        "text": f"""
        I asked you for a random number between 1 and 100 and you gave me {{ q_{n-1}.answer }}. 
        Please give me a new random number.
        """
    }) for n in range(1,6)]
)

The `loop` method creates a list of questions with the scenarios added in.
Note that because we used single-braces for ease of referencing the piped question names we will see a warning that scenarios require double braces, in case we used the single braces inadvertently. We can ignore this message here, and confirm that our questions have been formatted as intended:

In [5]:
qq = q.loop(s)
qq

[Question('numerical', question_name = """q_1""", question_text = """
         I asked you for a random number between 1 and 100 and you gave me { q_0.answer }. 
         Please give me a new random number.
         """, min_value = 1, max_value = 100, answering_instructions = """The number must be an integer."""),
 Question('numerical', question_name = """q_2""", question_text = """
         I asked you for a random number between 1 and 100 and you gave me { q_1.answer }. 
         Please give me a new random number.
         """, min_value = 1, max_value = 100, answering_instructions = """The number must be an integer."""),
 Question('numerical', question_name = """q_3""", question_text = """
         I asked you for a random number between 1 and 100 and you gave me { q_2.answer }. 
         Please give me a new random number.
         """, min_value = 1, max_value = 100, answering_instructions = """The number must be an integer."""),
 Question('numerical', question_name = """q_4""",

We pass the list of questions to a `Survey` object as usual in order to administer them together.
Note that because we are piping answers into questions, the questions will automatically be administered in the order required by the piping. (If no piping or other survey rules are applied, questions are administered asychronously by default. Learn more about applying [survey rules and logic](https://docs.expectedparrot.com/en/latest/surveys.html).)

We can re-inspect the questions that are now in a survey:

In [6]:
survey = Survey(questions = [q_0] + qq)
survey

,question_name,question_text,min_value,max_value,answering_instructions,question_type
0,q_0,Please give me a random number.,1,100,The number must be an integer.,numerical
1,q_1,I asked you for a random number between 1 and 100 and you gave me { q_0.answer }. Please give me a new random number.,1,100,The number must be an integer.,numerical
2,q_2,I asked you for a random number between 1 and 100 and you gave me { q_1.answer }. Please give me a new random number.,1,100,The number must be an integer.,numerical
3,q_3,I asked you for a random number between 1 and 100 and you gave me { q_2.answer }. Please give me a new random number.,1,100,The number must be an integer.,numerical
4,q_4,I asked you for a random number between 1 and 100 and you gave me { q_3.answer }. Please give me a new random number.,1,100,The number must be an integer.,numerical
5,q_5,I asked you for a random number between 1 and 100 and you gave me { q_4.answer }. Please give me a new random number.,1,100,The number must be an integer.,numerical


Next we select some models to generate responses (see our [models pricing page](https://www.expectedparrot.com/home/pricing) for details on available models and documentation on specifying [model parameters](https://docs.expectedparrot.com/en/latest/language_models.html)):

In [7]:
from edsl import Model, ModelList

m = ModelList([
    Model("gemini-2.5-flash", service_name = "google"),
    Model("gpt-4o", service_name = "openai")
    # etc.
])

We run the survey by adding the models and then calling the `run()` method on it:

In [8]:
results = survey.by(m).run()

We can see a list of the columns of the dataset of `Results` that has been generated:

In [9]:
results.columns

,0
0,agent.agent_index
1,agent.agent_instruction
2,agent.agent_name
3,answer.q_0
4,answer.q_1
5,answer.q_2
6,answer.q_3
7,answer.q_4
8,answer.q_5
9,cache_keys.q_0_cache_key


All of these components can be analyzed in a variety of [built-in methods for working with results](https://docs.expectedparrot.com/en/latest/results.html).
Here we create a table of responses, together with the question prompts to verify that the piping worked:

In [10]:
(
    results
    .select(
        "model",
        "prompt.q_0_user_prompt", "q_0",
        "prompt.q_1_user_prompt", "q_1",
        "prompt.q_2_user_prompt", "q_2",
        "prompt.q_3_user_prompt", "q_3",
        "prompt.q_4_user_prompt", "q_4",
        "prompt.q_5_user_prompt", "q_5"
    )
)

,model.model,prompt.q_0_user_prompt,answer.q_0,prompt.q_1_user_prompt,answer.q_1,prompt.q_2_user_prompt,answer.q_2,prompt.q_3_user_prompt,answer.q_3,prompt.q_4_user_prompt,answer.q_4,prompt.q_5_user_prompt,answer.q_5
0,gemini-2.5-flash,Please give me a random number. Minimum answer value: 1 Maximum answer value: 100 The number must be an integer.,67,I asked you for a random number between 1 and 100 and you gave me { q_0.answer }. Please give me a new random number. Minimum answer value: 1 Maximum answer value: 100 The number must be an integer.,42,I asked you for a random number between 1 and 100 and you gave me { q_1.answer }. Please give me a new random number. Minimum answer value: 1 Maximum answer value: 100 The number must be an integer.,42,I asked you for a random number between 1 and 100 and you gave me { q_2.answer }. Please give me a new random number. Minimum answer value: 1 Maximum answer value: 100 The number must be an integer.,42,I asked you for a random number between 1 and 100 and you gave me { q_3.answer }. Please give me a new random number. Minimum answer value: 1 Maximum answer value: 100 The number must be an integer.,42,I asked you for a random number between 1 and 100 and you gave me { q_4.answer }. Please give me a new random number. Minimum answer value: 1 Maximum answer value: 100 The number must be an integer.,97
1,gpt-4o,Please give me a random number. Minimum answer value: 1 Maximum answer value: 100 The number must be an integer.,1,I asked you for a random number between 1 and 100 and you gave me { q_0.answer }. Please give me a new random number. Minimum answer value: 1 Maximum answer value: 100 The number must be an integer.,1,I asked you for a random number between 1 and 100 and you gave me { q_1.answer }. Please give me a new random number. Minimum answer value: 1 Maximum answer value: 100 The number must be an integer.,1,I asked you for a random number between 1 and 100 and you gave me { q_2.answer }. Please give me a new random number. Minimum answer value: 1 Maximum answer value: 100 The number must be an integer.,1,I asked you for a random number between 1 and 100 and you gave me { q_3.answer }. Please give me a new random number. Minimum answer value: 1 Maximum answer value: 100 The number must be an integer.,1,I asked you for a random number between 1 and 100 and you gave me { q_4.answer }. Please give me a new random number. Minimum answer value: 1 Maximum answer value: 100 The number must be an integer.,1


## Adding question memory
Re: survey rules mentioned above--here we automatically add a memory of *all* prior questions to each new question, e.g., to see how this may impact responses:

In [11]:
results_memory = survey.set_full_memory_mode().by(m).run()

In [12]:
results_memory.select("model", "q_0", "q_1", "q_2", "q_3", "q_4", "q_5")

,model.model,answer.q_0,answer.q_1,answer.q_2,answer.q_3,answer.q_4,answer.q_5
0,gemini-2.5-flash,67,92,42,31,85,17
1,gpt-4o,1,1,1,1,1,1


## Posting to Coop
Coop is a platform for posting and sharing AI-based research.
It is fully integrated with EDSL and free to use.
Learn more about [how it works](https://docs.expectedparrot.com/en/latest/coop.html) or create an account: [https://www.expectedparrot.com/login](https://www.expectedparrot.com/login).

In the examples above, results generated using [remote inference](https://docs.expectedparrot.com/en/latest/remote_inference.html) (run at the Expected Parrot server) were automatically posted to Coop (see links to results).

Here we show how to manually post any local content to Coop, such as this notebook:

In [ ]:
from edsl import Notebook

nb = Notebook(path = "looping_and_piping.ipynb")

nb.push(
    description = "Simultaneous looping and piping", 
    alias = "looping-piping",
    visibility = "public"
)

Content posted to Coop can be modified from your workspace or at the web app at any time.